# グラフェンの変形をできるだけおこさなくする。

(2024-11-05)

graphene-studio で生成した炭素配置をできるだけ維持するように、分子内の拘束条件(constraint)を減らすかわりに、原子を正しい位置に束縛(restraint)する。

## 注意

水分子モデルが TIP4P になっている。


## 1. 水和させる

graphene-studio で生成した構造の 1 つを選び、ファイル`graphene.gro`にコピーする。

その構造を水和させる。あらかじめ、セルの大きさは z 方向に 2 倍になっている。


In [5]:
! cp diamond_90.itp graphene.itp
! cp diamond_90.gro graphene.gro
! gmx solvate -cs tip4p -cp graphene.gro -o solvated.gro -p solvated-fullfix.top 


                 :-) GROMACS - gmx solvate, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx solvate -cs tip4p -cp graphene.gro -o solvated.gro -p solvated-fullfix.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be di

## 2. 炭素原子をゆるく固定する。

(手順は`HMTA/`からもってきた。)

炭素を定位置に固定(restraint)する外場は`graphene-restraints.itp`ファイルに書く。こんな感じの拘束条件を原子の個数だけ書きならべる。

```shell
[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz
   1    1       10000       10000       10000
   2    1       10000       10000       10000
...
```

原子の個数だけバネが必要なので長大なデータになる。自動生成させるのが良い。
以下のコードで、position_restraints を自動生成する。


In [ ]:
from graphenestudio.gromacs import read_gro

# 原子数を読みとるだけのためにgraphene.groを読みこんでいる。
with open("graphene.gro") as f:
    for frame in read_gro(f):
        Natom = frame.residue_id.shape[0]
        break

# 原子の個数だけバネをつける。
with open("graphene-restraints.itp", "w") as f:
    print(
        """[ position_restraints ]
; 炭素を固定する。
;  i funct       fcx        fcy        fcz""",
        file=f,
    )
    for i in range(Natom):
        print(f"{i+1} 1 10000 10000 10000", file=f)

あるいはコマンドラインで


In [6]:
! python restrain.py < graphene.gro > graphene-restraints.itp

## 3. MD の準備

- `-r`オプションで固定先の座標を指定する。錨をおろす位置は継続計算でも変更してはいけない。
- シミュレーションの条件は`initial-solvated-fullfix.mdp`に書かれている。ここでは、圧力制御を`berendsen`とし、`semiisotropic`な変形を許す。
- `solvated-fullfix.top`の中で、上で作った`graphene-restraints.itp`が読みこまれることで、どの原子を固定するかが指示されている。
- 原子数があわないというエラーが出る場合は、`solvated-fullfix.top`ファイルに書きこまれた水和水分子数が正しいかどうかを確認する。
- 水分子モデルを変更する場合にも`solvated-fullfix.top`を編集する。


In [10]:
! gmx grompp -maxwarn 2 -f initial-solvated-fullfix.mdp -p solvated-fullfix.top -o 00001.tpr -c solvated.gro -r solvated.gro

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx grompp -maxwarn 2 -f initial-solvated-fullfix.mdp -p solvated-fullfix.top -o 00001.tpr -c solvated.gro -r solvated.gro

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'cpp'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'

WARNING 1 [file initial-solvated-fullfix.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to -1611782668

Generated 15 of the 15 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'GRPH'

Excluding 1 bonded neighbours molecule type 'w

## 4. 初回の MD 実行


In [11]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00001 -c 00001-last.gro

                  :-) GROMACS - gmx mdrun, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx mdrun -notunepme -nt 8 -deffnm 00001 -c 00001-last.gro

Reading file 00001.tpr, VERSION 2024.2-Homebrew (single precision)
Changing nstlist from 10 to 100, rlist from 0.9 to 0.904

NOTE: Periodic molecules are present in this system. Because of this, the domain decomposition algorithm cannot easily determine the minimum cell size that it requires for treating bonded interactions. Instead, domain decomposition will assume that half the non-bonded cut-off will be a suitable lower bound.
Using 1 MPI thread
Using 8 OpenMP threads 

starting mdrun 'solvated graphene in water'
50000 steps,      5.0 ps.

Writing final coordinates.

               Core t (s)   Wall t (s)        (%)
       Time:     1490.860      186.358      800.0
    

In [12]:
! make 00001-0.gro

prev=`expr 00001 - 1`; prev=`printf '%05d' $prev`; echo $prev; \
	if [ -e $prev.part*.trr ] ; \
		then echo 0 | gmx trjconv -f $prev.part*.trr -s 00001.tpr -o 00001-0.gro; \
	elif [ -e 00001.trr ] ; \
		then echo 0 | gmx trjconv -f 00001.trr -s 00001.tpr -o 00001-0.gro; \
	fi
00000
                 :-) GROMACS - gmx trjconv, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx trjconv -f 00001.trr -s 00001.tpr -o 00001-0.gro

Note that major changes are planned in future for trjconv, to improve usability and utility.
Will write gro: Coordinate file in Gromos-87 format
Reading file 00001.tpr, VERSION 2024.2-Homebrew (single precision)
Reading file 00001.tpr, VERSION 2024.2-Homebrew (single precision)
Select group for output
Group     0 (         System) has  2800 elements
Group     1 (          Other) has  2800 elements
Gr

## 5. 継続計算

継続する場合も、炭素原子固定位置を指示する部分はそのまま使う。

- `continue-solvated-fullfix.mdp`での変更点は
  1. 温度を 270 K に。
  2. extended_simulation に関する設定
     - `gen_vel                  = no`
     - `continuation             = yes`
  3. dt をまともな数字(0.001 ps)に。
  4. 圧力を 1 気圧に設定。

`Nose-Hoover`では異常に分子運動が抑えられてしまうので、やむをえず`v-rescale`を継続使用している。


`diff`コマンドで設定の違いを比較する。


In [17]:
! diff initial-solvated-fullfix.mdp continue-solvated-fullfix.mdp

8c8
< dt                       = 0.0001     ; MD計算のΔt (ps)。
---
> dt                       = 0.001     ; MD計算のΔt (ps)。
42c42
< ref_t                    = 300.0       ; 系の温度。単位はK。
---
> ref_t                    = 250.0       ; 系の温度。単位はK。
49c49
< ref_p                    = 1    1000     ; 系の圧力。単位はbar。Pcoupl = no の場合は無効。
---
> ref_p                    = 1    1    ; 系の圧力。単位はbar。Pcoupl = no の場合は無効。
52,53c52,53
< gen_vel                  = yes
< continuation             = no
---
> gen_vel                  = no
> continuation             = yes


In [16]:
! gmx grompp -maxwarn 2 -f continue-solvated-fullfix.mdp -p solvated-fullfix.top -o 00002.tpr -c 00001-last.gro -r solvated.gro

                 :-) GROMACS - gmx grompp, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx grompp -maxwarn 2 -f continue-solvated-fullfix.mdp -p solvated-fullfix.top -o 00002.tpr -c 00001-last.gro -r solvated.gro

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'cpp'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'

WARNING 1 [file continue-solvated-fullfix.mdp]:
  The Berendsen barostat does not generate any strictly correct ensemble,
  and should not be used for new production simulations (in our opinion).
  We recommend using the C-rescale barostat instead.

Setting the LD random seed to 381681630

Generated 15 of the 15 non-bonded parameter combinations

Excluding 3 bonded neighbours molecule type 'GRPH'

Excluding 1 bonded neighbours molecule type 

In [18]:
! gmx mdrun -notunepme  -nt 8 -deffnm 00002 -c 00002-last.gro

                  :-) GROMACS - gmx mdrun, 2024.2-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2024.2/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2024.2
Working dir:  /Users/matto/Unison/github/graphene-studio
Command line:
  gmx mdrun -notunepme -nt 8 -deffnm 00002 -c 00002-last.gro


Back Off! I just backed up 00002.log to ./#00002.log.1#
Reading file 00002.tpr, VERSION 2024.2-Homebrew (single precision)
Changing nstlist from 10 to 80, rlist from 0.901 to 1.013

NOTE: Periodic molecules are present in this system. Because of this, the domain decomposition algorithm cannot easily determine the minimum cell size that it requires for treating bonded interactions. Instead, domain decomposition will assume that half the non-bonded cut-off will be a suitable lower bound.
Using 1 MPI thread
Using 8 OpenMP threads 


Back Off! I just backed up 00002.trr to ./#00002.trr.1#

Back Off! I just backed up 00002.edr to ./#00002.edr.1#
starting mdrun 'solvated graphe

In [26]:
! make 00002-0.gro

make: `00002-0.gro' is up to date.


## リングの可視化

analice2 を使う。


In [25]:
! analice2 00002-0.gro -O OW -H HW[12] -f rings[yaplot] > 00002-0.yap

INFO:  Largest ring: 8.
INFO:Output:None
INFO:  file number range: 0:1000000:1
INFO:  frame number range: 0:1000000:1
INFO:00002-0.gro
INFO:  File name: 00002-0.gro
INFO:Frame: 0
INFO:  
INFO:  Command line: /Users/matto/Unison/github/graphene-studio/.venv/bin/analice2 00002-0.gro -O OW -H HW[12] -f rings[yaplot]
INFO:Cell dimension:
INFO:  a = 2.27569
INFO:  b = 2.27569
INFO:  c = 5.00919
INFO:  A = 90.0
INFO:  B = 90.0
INFO:  C = 90.0
INFO:Bond length (estim.): 0.25996999945625254
INFO:Target Density: 0.5899402226645347
INFO:Original Density: 0.5899402226645347
INFO:Bond length (scaled, nm): 0.25996999945625254
INFO:Stage1: Preparation.
INFO:  Number of water molecules: 512
INFO:  Number of water nodes: 476
INFO:Cell dimension:
INFO:  a = 2.27569
INFO:  b = 2.27569
INFO:  c = 5.00919
INFO:  A = 90.0
INFO:  B = 90.0
INFO:  C = 90.0
INFO:Stage1: end.
INFO:Stage1: 1 ms
INFO:Hook2: Show rings.
INFO:Render: Rendering with Yaplot.
INFO:Render: end.
INFO:Render: 2 ms
INFO:Hook2: end.
INFO:H